In [1]:
# Dependencies
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
import requests
import numpy as np
import time
import json
pd.options.display.max_rows = 999

In [2]:
# Create Manager Dataframe to link Manager Name and Current Team Name.
# Had to hardcode manager name because league settings on this are private. 
# If order changes, this section will need to be revisited.

managers_url = 'https://football.fantasysports.yahoo.com/f1/528229/teams'
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless = True)
browser.visit(managers_url)
managers_html = browser.html
managers_soup = BeautifulSoup(managers_html, "html.parser")
managers_table = managers_soup.findAll("tbody")[1]
managers = managers_table.findAll("tr")
team_names = []
for m in managers:
    manager = m.findAll('td')
    team_name = manager[0].text
    team_names.append(team_name)
print (team_names)
manager_list = []
for x in np.arange(len(team_names)):
    manager = input("Who is the manager for " + team_names[x] + "? ")
    manager_list.append({"team_name": team_names[x], "manager": manager})
manager_df = pd.DataFrame(manager_list)
manager_df.head(12)

["2 Gurley's 1 Cup", "Chi ShingT's Team", "Ching ShiT's Team", 'Cohen for Three', 'Cry me a Philip', 'DougTrio', 'FirstRoundFlops', 'Freeman 4 3...Losses', 'G', 'Mitch Please', 'Nags', 'wRonNgfulTermination']
Who is the manager for 2 Gurley's 1 Cup? Matt
Who is the manager for Chi ShingT's Team? Chi Shing
Who is the manager for Ching ShiT's Team? Jake
Who is the manager for Cohen for Three? Sean
Who is the manager for Cry me a Philip? Evan
Who is the manager for DougTrio? Ron
Who is the manager for FirstRoundFlops? Rajiv
Who is the manager for Freeman 4 3...Losses? Dai
Who is the manager for G? Jiwei
Who is the manager for Mitch Please? Andrew
Who is the manager for Nags? Ryan
Who is the manager for wRonNgfulTermination? Doug


,manager,team_name
0,Matt,2 Gurley's 1 Cup
1,Chi Shing,Chi ShingT's Team
2,Jake,Ching ShiT's Team
3,Sean,Cohen for Three
4,Evan,Cry me a Philip
5,Ron,DougTrio
6,Rajiv,FirstRoundFlops
7,Dai,Freeman 4 3...Losses
8,Jiwei,G
9,Andrew,Mitch Please


In [3]:
# Import draft results. This will serve as the starting point for each scrape
# Todd Gurley II's and Dalvin Cook's values were swapped at draft time, so their values are hardcoded.

executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless = True)
draft_url = "https://football.fantasysports.yahoo.com/f1/528229/draftresults"
browser.visit(draft_url)
draft_html = browser.html
draft_soup = BeautifulSoup(draft_html, "html.parser")
draft_table = draft_soup.findAll("tbody")[1]
draft_picks = draft_table.findAll("tr")
draft_pick_list = []
for picks in draft_picks:
    pick = picks.findAll("td")
    name = pick[1].text.split(" (")[0]
    team = pick[1].text.split(" (")[1].split(" - ")[0]
    pos = pick[1].text.split(" (")[1].split(" - ")[1].split(")")[0]
    draft_pick_dict = {
        "name": name,
        "NFLTeam": team,
        "Pos": pos,
        "cost": int(pick[2].text[1:]),
        "team": pick[3].text
    }
    draft_pick_list.append(draft_pick_dict)
for player in draft_pick_list:
    if player['name'] == "Todd Gurley II":
        player["cost"] = 40
    if player["name"] == "Dalvin Cook":
        player["cost"] = 64

In [4]:
# Create the draft dataframe, as well as keeper column and manager column

draft_df = pd.DataFrame(draft_pick_list)
draft_df['keeper'] = ""
draft_df["manager"] = ""
draft_df.head(10)

,NFLTeam,Pos,cost,name,team,keeper,manager
0,Dal,QB,6,Dak Prescott,Ching ShiT's Team,,
1,Pit,WR,81,Antonio Brown,Chi ShingT's Team,,
2,NE,TE,64,Rob Gronkowski,Freeman 4 3...Losses,,
3,Jax,RB,60,Leonard Fournette,Cohen for Three,,
4,Ind,WR,51,T.Y. Hilton,Cry me a Philip,,
5,NYG,RB,74,Saquon Barkley,wRonNgfulTermination,,
6,Buf,RB,29,LeSean McCoy,Nags,,
7,LAC,QB,11,Philip Rivers,Mitch Please,,
8,Hou,RB,28,Lamar Miller,Cry me a Philip,,
9,Cin,WR,67,A.J. Green,Ching ShiT's Team,,


In [5]:
# Import Keeper CSV
# B denotes keeper for just this past year.
# AB denotes keeper for past two years.

keeper_df = pd.read_csv('keeper.csv')
keeper_df.head(15)

,player,duration
0,Dalvin Cook,B
1,Todd Gurley II,AB
2,Adam Thielen,B
3,DeAndre Hopkins,B
4,Melvin Gordon III,AB
5,Davante Adams,AB
6,Kenyan Drake,B
7,Golden Tate,AB
8,Corey Clement,B
9,Devonta Freeman,AB


In [6]:
# merge keeper dataframe with draft dataframe to include keepers into draft dataframe

for i, d in draft_df.iterrows():
    for index, keeper in keeper_df.iterrows():
        if d['name'] == keeper['player']:
            draft_df.at[i,'keeper'] = keeper['duration']
            
draft_df.head(10)

,NFLTeam,Pos,cost,name,team,keeper,manager
0,Dal,QB,6,Dak Prescott,Ching ShiT's Team,,
1,Pit,WR,81,Antonio Brown,Chi ShingT's Team,,
2,NE,TE,64,Rob Gronkowski,Freeman 4 3...Losses,,
3,Jax,RB,60,Leonard Fournette,Cohen for Three,,
4,Ind,WR,51,T.Y. Hilton,Cry me a Philip,,
5,NYG,RB,74,Saquon Barkley,wRonNgfulTermination,,
6,Buf,RB,29,LeSean McCoy,Nags,,
7,LAC,QB,11,Philip Rivers,Mitch Please,,
8,Hou,RB,28,Lamar Miller,Cry me a Philip,,
9,Cin,WR,67,A.J. Green,Ching ShiT's Team,,


In [7]:
# merge manager name with draft picks dataframe

for i, d in draft_df.iterrows():
    for index, team in manager_df.iterrows():
        if d["team"] == team["team_name"]:
            draft_df.at[i, "manager"] = team["manager"]
draft_df.head(15)

,NFLTeam,Pos,cost,name,team,keeper,manager
0,Dal,QB,6,Dak Prescott,Ching ShiT's Team,,Jake
1,Pit,WR,81,Antonio Brown,Chi ShingT's Team,,Chi Shing
2,NE,TE,64,Rob Gronkowski,Freeman 4 3...Losses,,Dai
3,Jax,RB,60,Leonard Fournette,Cohen for Three,,Sean
4,Ind,WR,51,T.Y. Hilton,Cry me a Philip,,Evan
5,NYG,RB,74,Saquon Barkley,wRonNgfulTermination,,Doug
6,Buf,RB,29,LeSean McCoy,Nags,,Ryan
7,LAC,QB,11,Philip Rivers,Mitch Please,,Andrew
8,Hou,RB,28,Lamar Miller,Cry me a Philip,,Evan
9,Cin,WR,67,A.J. Green,Ching ShiT's Team,,Jake


In [8]:
# Create a list of transactions by scraping yahoo fantasy sheet for all transactions.
# Value returned as multiple lines so had to split out and remove any injury statuses.
# Placed values in dictionary and merged all the dictionaries.
# Entries go from newest to oldest.

transaction_pages = np.arange(5)*25
transaction_list = []
for page in transaction_pages:
    try:
        transaction_url = 'https://football.fantasysports.yahoo.com/f1/528229/transactions?transactionsfilter=all&count=' + str(page)
        executable_path = {'executable_path': 'chromedriver.exe'}
        browser = Browser('chrome', **executable_path, headless = True)
        browser.visit(transaction_url)
        transaction_html = browser.html
        transaction_soup = BeautifulSoup(transaction_html, "html.parser")
        transaction_table = transaction_soup.findAll("tbody")[1]
        transactions = transaction_table.findAll("tr")
        for t in transactions:
            transaction = t.findAll('td')
            transaction_info = transaction[1].text.splitlines()
            team_info = transaction[2].text.splitlines()
            try:
                try:
                    transaction_info.remove("Q")
                except:
                    pass
                try:
                    transaction_info.remove("D")
                except:
                    pass
                try:
                    transaction_info.remove("O")
                except:
                    pass
                try:
                    transaction_info.remove("NA")
                except:
                    pass
                if transaction_info[3].strip()[0] == "$":
                    price = int(transaction_info[3].strip().split("  ")[0][1:])
                else:
                    price = int(0)
                tm = team_info[3].strip()
                for i, t in manager_df.iterrows():
                    if t['team_name'] == tm:
                        transaction_dict = {
                            "add": transaction_info[1].strip(),
                            "nfl_team": transaction_info[2].strip().split(" - ")[0],
                            "position": transaction_info[2].strip().split(" - ")[1],
                            "price":price,
                            "drop":transaction_info[4].strip(),
                            "ffteam": t['manager'],
                            "type": "waiver"
                        }
                        transaction_list.append(transaction_dict)
            except:
                pass
    except:
        pass



In [9]:
# manually code in trade. This will be for reference only.
# Index manually coded as well so that it will appear in the correct order if the entire dataframe is viewed.
# Created index  for draft position for traded players.

trade_list = []
trade00a = {"index": 53.3,
           "add": "Marquise Goodwin",
           "nfl_team": "SF",
           "position": "WR",
           "price":5,
           "drop":"Adrian Peterson",
           "ffteam":"Matt",
           "type": "trade"}
trade00b = {"index": 53.6,
           "add": "Adrian Peterson",
           "nfl_team": "WAS",
           "position": "RB",
           "price":12,
           "drop":"Marquise Goodwin",
           "ffteam":"Jake",
           "type": "trade"}
trade00c = {"index": 66.3,
           "add": "Sony Michel",
           "nfl_team": "NE",
           "position": "RB",
           "price":7,
           "drop":"Geronimo Allison",
           "ffteam":"Ron",
           "type": "trade"}
trade00d = {"index": 66.6,
           "add": "Geronimo Allison",
           "nfl_team": "GB",
           "position": "WR",
           "price":0,
           "drop":"Sony Michel",
           "ffteam":"Andrew",
           "type": "trade"}
trade_list.append(trade00a)
trade_list.append(trade00b)
trade_list.append(trade00c)
trade_list.append(trade00d)


In [10]:
# flipped data so that oldest appears first and newest appears last for all waiver transactions, not trades.

transaction_df = pd.DataFrame(transaction_list)
transaction_df = transaction_df.reindex(index=transaction_df.index[::-1]).reset_index(drop=True)
transaction_df = transaction_df.drop([48, 49]).reset_index(drop=True)
transaction_df.head(200)

,add,drop,ffteam,nfl_team,position,price,type
0,Nyheim Hines,Ronald Jones II,Andrew,Ind,RB,0,waiver
1,Theo Riddick,Darren Sproles,Chi Shing,Det,RB,0,waiver
2,Chandler Catanzaro,Greg Zuerlein,Jiwei,TB,K,0,waiver
3,Matt Prater,Corey Grant,Dai,Det,K,0,waiver
4,Christian Kirk,Mark Walton,Dai,Ari,WR,0,waiver
5,Ronald Jones II,Jamison Crowder,Dai,TB,RB,0,waiver
6,Jordy Nelson,Tyrod Taylor,Jiwei,Oak,WR,7,waiver
7,Ryan Tannehill,Doug Martin,Doug,Mia,QB,6,waiver
8,Baker Mayfield,Austin Ekeler,Sean,Cle,QB,6,waiver
9,Chris Ivory,Spencer Ware,Jiwei,Buf,RB,5,waiver


In [11]:
# merge transactions and trades. trade will sent index to the index column in the dataframe

trade_df = pd.DataFrame(trade_list)
trade_df.set_index('index', inplace=True)
trade_df.head()
transaction_df = pd.concat([transaction_df, trade_df])

In [12]:
# re-order index so that trade appears in the correct order
transaction_df = transaction_df.sort_index()
transaction_df.head(200)

,add,drop,ffteam,nfl_team,position,price,type
0.0,Nyheim Hines,Ronald Jones II,Andrew,Ind,RB,0,waiver
1.0,Theo Riddick,Darren Sproles,Chi Shing,Det,RB,0,waiver
2.0,Chandler Catanzaro,Greg Zuerlein,Jiwei,TB,K,0,waiver
3.0,Matt Prater,Corey Grant,Dai,Det,K,0,waiver
4.0,Christian Kirk,Mark Walton,Dai,Ari,WR,0,waiver
5.0,Ronald Jones II,Jamison Crowder,Dai,TB,RB,0,waiver
6.0,Jordy Nelson,Tyrod Taylor,Jiwei,Oak,WR,7,waiver
7.0,Ryan Tannehill,Doug Martin,Doug,Mia,QB,6,waiver
8.0,Baker Mayfield,Austin Ekeler,Sean,Cle,QB,6,waiver
9.0,Chris Ivory,Spencer Ware,Jiwei,Buf,RB,5,waiver


In [13]:
# run through total transaction dataframe and draft dataframe swapping out players who are dropped for players who are added.
# additionally, transacation costs, and player information added to draft dataframe
# if the index matches the draft_df for a particular player, then only the manager's name is swapped. This is hard-coded.
for x, t in transaction_df.iterrows():
    for i, d in draft_df.iterrows():
        if t['type'] == "waiver":
            if d['name'] == t['drop']:
                draft_df.at[i, 'name'] = t['add']
                draft_df.at[i, 'NFLTeam'] = t['nfl_team']
                draft_df.at[i, 'Pos'] = t['position']
                draft_df.at[i, 'cost'] = t['price']
                draft_df.at[i, 'keeper'] = ""
        if t["type"] == "trade":
            trade_index = draft_df.index[draft_df['name']==t['add']]
            draft_df.at[trade_index, "manager"] = t['ffteam']
roster_df = draft_df


In [14]:
roster_df.head(200)

,NFLTeam,Pos,cost,name,team,keeper,manager
0,Dal,QB,6,Dak Prescott,Ching ShiT's Team,,Jake
1,Pit,WR,81,Antonio Brown,Chi ShingT's Team,,Chi Shing
2,NE,TE,64,Rob Gronkowski,Freeman 4 3...Losses,,Dai
3,Jax,RB,60,Leonard Fournette,Cohen for Three,,Sean
4,Ind,WR,51,T.Y. Hilton,Cry me a Philip,,Evan
5,NYG,RB,74,Saquon Barkley,wRonNgfulTermination,,Doug
6,Buf,RB,29,LeSean McCoy,Nags,,Ryan
7,LAC,QB,11,Philip Rivers,Mitch Please,,Andrew
8,Hou,RB,28,Lamar Miller,Cry me a Philip,,Evan
9,Cin,WR,67,A.J. Green,Ching ShiT's Team,,Jake


In [15]:
# reset the team names so that manager and most recent team name are matching again.
# this is done primarily for the traded player

for i, r in roster_df.iterrows():
    for index, t in manager_df.iterrows():
        if r['manager'] == t['manager']:
            roster_df.at[i, "team"] = t["team_name"]
            
roster_df.head(15)

,NFLTeam,Pos,cost,name,team,keeper,manager
0,Dal,QB,6,Dak Prescott,Ching ShiT's Team,,Jake
1,Pit,WR,81,Antonio Brown,Chi ShingT's Team,,Chi Shing
2,NE,TE,64,Rob Gronkowski,Freeman 4 3...Losses,,Dai
3,Jax,RB,60,Leonard Fournette,Cohen for Three,,Sean
4,Ind,WR,51,T.Y. Hilton,Cry me a Philip,,Evan
5,NYG,RB,74,Saquon Barkley,wRonNgfulTermination,,Doug
6,Buf,RB,29,LeSean McCoy,Nags,,Ryan
7,LAC,QB,11,Philip Rivers,Mitch Please,,Andrew
8,Hou,RB,28,Lamar Miller,Cry me a Philip,,Evan
9,Cin,WR,67,A.J. Green,Ching ShiT's Team,,Jake


In [16]:
# grab the unique team names

ff_team_names = list(manager_df.team_name)
print(ff_team_names)

["2 Gurley's 1 Cup", "Chi ShingT's Team", "Ching ShiT's Team", 'Cohen for Three', 'Cry me a Philip', 'DougTrio', 'FirstRoundFlops', 'Freeman 4 3...Losses', 'G', 'Mitch Please', 'Nags', 'wRonNgfulTermination']


In [17]:
# create a list of all rosters sorted by cost of player for each team

roster_df = roster_df.sort_values(by=['cost'], ascending=False)
roster_df = roster_df[['name', 'NFLTeam', 'Pos', 'cost', 'keeper', 'team', 'manager']]
for ff in ff_team_names:
    print(roster_df.loc[roster_df['team'] == ff])
    print("------------------------------------------------------------------------------------")

                 name NFLTeam  Pos  cost keeper              team manager
157       Dalvin Cook     Min   RB    64      B  2 Gurley's 1 Cup    Matt
158    Todd Gurley II     LAR   RB    40     AB  2 Gurley's 1 Cup    Matt
15     Alshon Jeffery     Phi   WR    40         2 Gurley's 1 Cup    Matt
33      Jarvis Landry     Cle   WR    38         2 Gurley's 1 Cup    Matt
156      Adam Thielen     Min   WR    28      B  2 Gurley's 1 Cup    Matt
64     Marcus Mariota     Ten   QB    22         2 Gurley's 1 Cup    Matt
57   Emmanuel Sanders     Den   WR    18         2 Gurley's 1 Cup    Matt
68        Jordan Reed     Was   TE    10         2 Gurley's 1 Cup    Matt
63        Matt Breida      SF   RB     8         2 Gurley's 1 Cup    Matt
92         Josh Rosen     Ari   QB     5         2 Gurley's 1 Cup    Matt
179  Marquise Goodwin      SF   WR     5      B  2 Gurley's 1 Cup    Matt
71         Jared Cook     Oak   TE     3         2 Gurley's 1 Cup    Matt
121       Aaron Jones      GB   RB    

In [18]:
# Save data as CSV
roster_df.to_csv("current_roster.csv")